In [1]:
import json
import os
from enum import Enum

In [2]:
class Oven:
    def __init__(self, start_temp : int, working_temps : list[int], operations : list[str]) -> None:
        self.start_temp = start_temp
        self.working_temps = working_temps
        self.operations = operations

class Operation:
    def __init__(self, name : str, timing : int) -> None:
        self.name = name
        self.timing = timing

class Series:
    def __init__(self, temperature : int, operations : list[Operation]) -> None:
        self.temperature = temperature
        self.operations = [Operation(**json.loads(json.dumps(op))) for op in operations]

class DayData:
    def __init__(self, ovens : list[Oven], series : list[Series]) -> None:
        self.ovens = [Oven(**json.loads(json.dumps(oven))) for oven in ovens]
        self.series = [Series(**json.loads(json.dumps(ser))) for ser in series]

class OPERS(Enum):
    prokat = -1
    kovka = -2
    otzhig = -3
    nagrev = -4

class OperationDto:
    def __init__(self, type, abs_start, abs_end) -> None:
        self.type = type
        self.abs_start = abs_start
        self.abs_end = abs_end

class SeriesDto:
    def __init__(self, operations : list[OperationDto]) -> None:
        self.operations = operations

In [3]:
dataset : list[DayData] = []
root = './train_dataset_train/train/'
# for _,_,files in os.walk(root):
#     for file in files:
#         dataset.append(DayData(**json.loads(open(root + file).read())))
path = 'mini-day1.json'
dataset.append(DayData(**json.loads(open(path).read())))


In [9]:
def first(src : list, cond):
    for i in src:
        if cond(i):
            return i
    return None

Предварительное распределение по печкам

In [11]:
def find_min(keys : list, src : dict) -> Oven:
    minv = 10e9
    min_obj = None
    for key in keys:
        if len(src[key]) < minv:
            minv = len(src[key])
            min_obj = key
    
    return min_obj



ovens = dataset[0].ovens
series = dataset[0].series

wasted_series = []
ovens_stream : dict[Oven, list[Series]] = {
    oven : [] for oven in ovens
}

for ser in series:
    same_opers_ovens = [oven for oven in ovens if \
                        len(set(oven.operations) & set([op.name for op in ser.operations])) == (len(set([op.name for op in ser.operations])) - 1)]
    

    #print([op.name for op in ser.operations], [oven.operations for oven in same_opers_ovens])
    if len(same_opers_ovens):

        same_temps_ovens = [oven for oven in same_opers_ovens if oven.start_temp == ser.temperature]
        if len(same_temps_ovens):
            min_filled_oven = find_min(same_temps_ovens, ovens_stream)
            ovens_stream[min_filled_oven].append(ser)
        else:
            min_filled_oven = find_min(same_opers_ovens, ovens_stream)
            ovens_stream[min_filled_oven].append(ser)
    else:
        wasted_series.append(ser)


# (temp, []), [(temp, [])]

def pretty_print(src):
    return [((key.start_temp, key.operations), [(ser.temperature, [(op.name, op.timing) for op in ser.operations])
      for ser in src[key]]) for key in src.keys() if 'prokat' in key.operations]

pretty_print(ovens_stream)

[((980, ['prokat', 'prokat']),
  [(1020, [('nagrev', 300), ('prokat', 15)]),
   (980, [('nagrev', 222), ('prokat', 15)]),
   (980, [('nagrev', 82), ('prokat', 15)])]),
 ((1100, ['prokat', 'prokat']),
  [(990, [('nagrev', 125), ('prokat', 15)]),
   (1030, [('nagrev', 45), ('prokat', 15)]),
   (1060, [('nagrev', 123), ('prokat', 15)])]),
 ((1130, ['prokat']),
  [(1040, [('nagrev', 213), ('prokat', 15)]),
   (950, [('nagrev', 87), ('prokat', 15)])]),
 ((1090, ['prokat']),
  [(1080, [('nagrev', 245), ('prokat', 15)]),
   (1050, [('nagrev', 171), ('prokat', 15)]),
   (1090, [('nagrev', 72), ('prokat', 15)])])]

Оптимизация потока серий в каждой печи

In [12]:
for oven in ovens_stream.keys():
    ovens_stream[oven].sort(key=lambda s: abs(s.temperature - oven.start_temp))


pretty_print(ovens_stream)

[((980, ['prokat', 'prokat']),
  [(980, [('nagrev', 222), ('prokat', 15)]),
   (980, [('nagrev', 82), ('prokat', 15)]),
   (1020, [('nagrev', 300), ('prokat', 15)])]),
 ((1100, ['prokat', 'prokat']),
  [(1060, [('nagrev', 123), ('prokat', 15)]),
   (1030, [('nagrev', 45), ('prokat', 15)]),
   (990, [('nagrev', 125), ('prokat', 15)])]),
 ((1130, ['prokat']),
  [(1040, [('nagrev', 213), ('prokat', 15)]),
   (950, [('nagrev', 87), ('prokat', 15)])]),
 ((1090, ['prokat']),
  [(1090, [('nagrev', 72), ('prokat', 15)]),
   (1080, [('nagrev', 245), ('prokat', 15)]),
   (1050, [('nagrev', 171), ('prokat', 15)])])]

In [14]:
def optimize_with_timing(oven_temp : int, series : list[Series]) -> list[SeriesDto]:
    curr_time = 0
    TIME_ELAPSED = 1440
    dto_series : list[SeriesDto] = []
    for ser in series:
        to_upload = True

        opers_temp : list[OperationDto] = []
        # Вручную первую операцию
        if curr_time + ser.operations[0].timing <=TIME_ELAPSED:
            opers_temp.append(OperationDto(ser.operations[0].name, curr_time, curr_time + ser.operations[0].timing))
            curr_time += ser.operations[0].timing

        if ser.temperature != oven_temp:
            curr_time += 120
            oven_temp = ser.temperature

        for i in range(1, len(ser.operations)):
            op = ser.operations[i]
            if curr_time + op.timing > TIME_ELAPSED:
                to_upload = False
                break
            if ser.operations[i - 1].name == op.name:
                if curr_time + 120 >= TIME_ELAPSED:
                    to_upload = False
                    break 
                opers_temp.append(OperationDto('progrev goev', curr_time, curr_time + 120))
                curr_time += 120
        
            opers_temp.append(OperationDto(op.name, curr_time, curr_time + op.timing))
            curr_time += op.timing

        if to_upload:
            dto_series.append(SeriesDto(opers_temp))
    return dto_series

Формат Данных

In [15]:
[[(op.name, op.timing) for op in ser.operations] for ser in ovens_stream[ovens[1]]]

[[('nagrev', 139), ('kovka', 20), ('kovka', 15)],
 [('nagrev', 232), ('kovka', 10), ('kovka', 10), ('kovka', 20)],
 [('nagrev', 296), ('kovka', 10), ('kovka', 10), ('kovka', 10)],
 [('nagrev', 135), ('kovka', 20), ('kovka', 20)]]

In [16]:
optimized_ovens_stream : dict[Oven, list[SeriesDto]]= {
    oven : [] for oven in ovens_stream.keys()
}
for oven in ovens_stream.keys():
    optimized_ovens_stream[oven] = optimize_with_timing(oven.start_temp, ovens_stream[oven])


oven_cnt = 1
with open('./out.txt', 'w') as w:
    for oven in optimized_ovens_stream.keys():
        series_cnt = 1
        w.write(f'###Oven N:{oven_cnt}; {oven.start_temp, oven.operations}###\n')
        for ser in optimized_ovens_stream[oven]:
            w.write(f'    Series {series_cnt} of {oven_cnt} oven\n')
            for op in ser.operations:
                w.write(f'        {op.type, op.abs_start, op.abs_end}\n')
            series_cnt += 1
        oven_cnt += 1

In [50]:
pretty_print(ovens_stream) #Данные в текстовом виде

[((980, ['prokat', 'prokat']),
  [(980, [('nagrev', 222), ('prokat', 15)]),
   (980, [('nagrev', 82), ('prokat', 15)]),
   (1020, [('nagrev', 300), ('prokat', 15)])]),
 ((1100, ['prokat', 'prokat']),
  [(1060, [('nagrev', 123), ('prokat', 15)]),
   (1030, [('nagrev', 45), ('prokat', 15)]),
   (990, [('nagrev', 125), ('prokat', 15)])]),
 ((1130, ['prokat']),
  [(1040, [('nagrev', 213), ('prokat', 15)]),
   (950, [('nagrev', 87), ('prokat', 15)])]),
 ((1090, ['prokat']),
  [(1090, [('nagrev', 72), ('prokat', 15)]),
   (1080, [('nagrev', 245), ('prokat', 15)]),
   (1050, [('nagrev', 171), ('prokat', 15)])])]

Преобразование и выпрямление данных

In [201]:
from typing import Any
def flatten(inp) -> list:
    CHAINED_OVENS = []
    def F(oven_pars):
        global CHAINED_OVENS
        for i in oven_pars:
            if type(i) in [list, tuple]:
                # print(1)
                # break
                F(i)
            else:
                CHAINED_OVENS = CHAINED_OVENS + [i]
                # print(CHAINED_OVENS)
        # CHAINED_OVENS = CHAINED_OVENS + [oven_pars]
    F(inp)
    


padding = 10
LABELS = []
for key, val in optimized_ovens_stream.items():
    a = list(set(key.operations))
    b = key.working_temps
    a = [en.value for en in OPERS for el in a if en.name == el] + [0] * (3 - len(a))
    LABELS.extend(a)
    LABELS.append(key.start_temp)
    LABELS.extend(b)
    # LABELS.extend(f'{a + [0] * (3 - len(a)), key.start_temp, b + [0] * (padding - len(b))}')
    print(f'{a + [0] * (3 - len(a)), key.start_temp, b + [0] * (padding - len(b))}')  ##читаемый  текст
    # print([en.value for en in OPERS for el in a if en.name == el] + [0] * (3 - len(a)), key.start_temp, b + [0] * (padding - len(b)))  ##рабочий   текст
    for ser in val:
        x = [[i.abs_start, i.abs_end] for i in ser.operations]
        LABELS.extend(x)
        # print(x)
    for _ in range(padding - val.__len__()):
        LABELS +=[ [0,0]]
    # print(val)
LABELS


([-1, 0, 0], 980, [1040, 970, 1180, 1030, 980, 0, 0, 0, 0, 0])
([-2, 0, 0], 1010, [960, 1180, 1110, 1010, 0, 0, 0, 0, 0, 0])
([-1, 0, 0], 1100, [1060, 1160, 1070, 1110, 950, 990, 1100, 0, 0, 0])
([-2, 0, 0], 1030, [1220, 1160, 1000, 1230, 1200, 1240, 1030, 0, 0, 0])
([-1, 0, 0], 1130, [1080, 1130, 1100, 0, 0, 0, 0, 0, 0, 0])
([-1, 0, 0], 1090, [1090, 1220, 1000, 1130, 1100, 1200, 1010, 1110, 1150, 0])
([-3, 0, 0], 1020, [960, 970, 1070, 1010, 1170, 1020, 990, 0, 0, 0])


[-1,
 0,
 0,
 980,
 1040,
 970,
 1180,
 1030,
 980,
 [0, 222],
 [222, 237],
 [237, 319],
 [319, 334],
 [334, 634],
 [754, 769],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 -2,
 0,
 0,
 1010,
 960,
 1180,
 1110,
 1010,
 [0, 139],
 [139, 159],
 [159, 279],
 [279, 294],
 [294, 526],
 [646, 656],
 [656, 776],
 [776, 786],
 [786, 906],
 [906, 926],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 -1,
 0,
 0,
 1100,
 1060,
 1160,
 1070,
 1110,
 950,
 990,
 1100,
 [0, 123],
 [243, 258],
 [258, 303],
 [423, 438],
 [438, 563],
 [683, 698],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 -2,
 0,
 0,
 1030,
 1220,
 1160,
 1000,
 1230,
 1200,
 1240,
 1030,
 [0, 207],
 [207, 227],
 [227, 347],
 [347, 362],
 [362, 482],
 [482, 492],
 [492, 741],
 [861, 881],
 [881, 1001],
 [1001, 1011],
 [1011, 1087],
 [1207, 1222],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 -1,
 0,
 0,
 1130,
 1080,
 1130,
 1100,
 [0, 213],
 [333, 348]

In [206]:
import numpy as np
# np.array(F(LABELS))
# CHAINED_OVENS
flatten(LABELS)  # flattten all data
labl = np.array(CHAINED_OVENS) # save data |always len is const  =5062
np.save('labls.npy', labl)
# CHAINED_OVENS
labl

array([-1,  0,  0, ...,  0,  0,  0])